![Python Logo](./img/python-logo-master-v3-TM-flattened_small.png)

# Financial Programming - IÉSEG, Lille - MBD 2017-2018
## Group Project - Financial Data

### Data Exploration

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from collections import defaultdict

# To be used to determine age & card duration
ref_date = datetime.datetime(year=2018, day=14, month=12)

**Input data**

In [2]:
# Each record describes static characteristics of an account
account = pd.read_csv('./data_berka/account.asc', sep=';')
order = pd.read_csv('./data_berka/order.asc', sep=';')
trans = pd.read_csv('./data_berka/trans.asc', sep=';', low_memory=False)
loan = pd.read_csv('./data_berka/loan.asc', sep=';')

disp = pd.read_csv('./data_berka/disp.asc', sep=';')
card = pd.read_csv('./data_berka/card.asc', sep=';')
client = pd.read_csv('./data_berka/client.asc', sep=';')
district = pd.read_csv('./data_berka/district.asc', sep=';')

#### Preprocessing the disp dataset

In [3]:
disp = disp.rename(columns={"type":"disp_type"})

#### Preprocessing the card dataset

In [4]:
card.issued = card.issued.apply(lambda x: (ref_date - pd.to_datetime(x[:6], format='%y%m%d')).days)

card = card.rename(columns={"type":"card_type", "issued":"days_since_card_issuance"})

#### Preprocessing the client dataset

In [5]:
# returns the nth digits from a number
def get_nth(number, nth):
    """nth should be an index or list of indexes for which to retrieve the digits"""
    return int("".join([val for i, val in enumerate(str(number)) if i in nth]))

# returns the month of birth_number.
def get_month(x):
    mth = get_nth(x, [2, 3])
    if mth > 50:
        return mth - 50
    else:
        return mth
    
# returns the gender by examining birth_number.
def birth_num_to_gender(x):
    mth = get_nth(x, [2, 3])
    if mth > 50:
        return 'F'
    else:
        return 'M'

# converts the birth_number into a date.
def birth_num_to_age(x):
    yr = get_nth(x, [0, 1]) + 1900
    mth = get_month(x)
    day = get_nth(x, [4, 5])
    return round((ref_date - datetime.datetime(yr, mth, day)).days/365, 2)

client["client_age"] = client.birth_number.apply(birth_num_to_age)
client["client_gender"] = client.birth_number.apply(birth_num_to_gender)
del client["birth_number"]

#### Preprocessing the district dataset

In [6]:
district = district.rename(columns={'A1':'district_id', 'A2':'district_name', 'A3':'region', 'A4':'num_inhabitants', 'A5':'num_munipalities_gt499',
 'A6':'num_munipalities_500to1999', 'A7':'num_munipalities_2000to9999', 'A8':'num_munipalities_gt10000',
 'A9':'num_cities', 'A10':'ratio_urban', 'A11':'average_salary', 'A12':'unemp_rate95', 'A13': 'unemp_rate96',
 'A14':'num_entrep_per1000', 'A15':'num_crimes95', 'A16':'num_crimes96'})

# Replacing "?" with proper missing values
district.unemp_rate95 = district.unemp_rate95.apply(lambda x: np.nan if x == "?" else x)
district.num_crimes95 = district.num_crimes95.apply(lambda x: np.nan if x == "?" else x)

# Converting the columns from string to floats
district.unemp_rate95 = district.unemp_rate95.apply(float)
district.num_crimes95 = district.num_crimes95.apply(float)

# Dealing with missing values (replace with mean of the region)
district.unemp_rate95 = district.groupby("region").unemp_rate95.transform(lambda x: x.fillna(x.mean()))
district.num_crimes95 = district.groupby("region").unemp_rate95.transform(lambda x: x.fillna(x.mean()))

In [7]:
# ref_start_date = datetime.datetime(year=1993, day=1, month=1)

# converts the encoded date into a normal date.
def get_date(x):
    yr = get_nth(x, [0, 1]) + 1900
    mth = get_month(x)
    day = get_nth(x, [4, 5])
    return datetime.datetime(yr, mth, day)

#### Preprocessing the account dataset

In [8]:
# Rename Account columns
account = account.rename(columns={'district_id': 'account_district_id','frequency': 'account_freq','date':'account_date_opened'})

# convert Account opening date to normal date
account['account_date_opened'] = account.account_date_opened.apply(get_date)

# convert account frequency to english
freq_options = defaultdict(lambda: 'UNKNOWN', 
                           {'POPLATEK MESICNE': 'MONTHLY', 
                            'POPLATEK TYDNE': 'WEEKLY', 
                            'POPLATEK PO OBRATU': 'TRANSACTION'})

account['account_freq'] = account.account_freq.apply(lambda x: freq_options[x])

#### Preprocessing the Order dataset

In [9]:
# Rename loan columns
order = order.rename(columns={'bank_to': 'order_bank_to', 'account_to':'order_account_to', 'amount':'order_amount',
                                   'k_symbol':'order_k_symbol'})

# convert order K-symbol to english
k_symbol_options = defaultdict(lambda: 'UNKNOWN', 
                           {'POJISTNE': 'INSURANCE_PAYMENT', 
                            'SIPO': 'HOUSEHOLD_PAYMENT', 
                            'LEASING': 'LEASING_PAYMENT',
                            'UVER':'LOAN_PAYMENT'})

order['order_k_symbol'] = order.order_k_symbol.apply(lambda x: k_symbol_options[x])

In [11]:
# Function to Recency, Frequency and Monetory aggregated values - Order data
def order_agg(row):
    row['freq_orders'] = row['order_id'].count()
    
    ins = row.loc[(row.order_k_symbol == 'INSURANCE_PAYMENT')]
    house = row.loc[(row.order_k_symbol == 'HOUSEHOLD_PAYMENT')]
    lease = row.loc[(row.order_k_symbol == 'LEASING_PAYMENT')]
    
    row['freq_order_insurance'] = ins['order_id'].count()
    row['freq_order_household'] = house['order_id'].count()
    row['freq_order_leasing'] = lease['order_id'].count()
    
    row['mon_order_insurance'] = ins['order_amount'].mean()
    row['mon_order_household'] = house['order_amount'].mean()
    row['mon_order_leasing'] = lease['order_amount'].mean()
    
    return row

order = order.groupby('account_id').apply(order_agg)

order_df = order.drop(['order_id', 'order_bank_to', 'order_account_to','order_amount','order_k_symbol'], axis=1).drop_duplicates()

order_df = order_df.fillna(0)

#### Preprocessing the Loan dataset

In [12]:
# Rename loan columns
loan = loan.rename(columns={'amount': 'loan_amount', 'duration':'loan_duration', 
                                  'payments':'monthly_loan_payment', 'status':'loan_status','date':'loan_date'})

# convert loan date to normal date
loan['loan_date'] = loan.loan_date.apply(get_date)

#### Preprocessing the Transaction dataset

In [13]:
# Rename transaction columns
trans = trans.rename(columns={'amount': 'trans_amount', 'balance':'balance_after_trans', 
                                    'bank':'trans_bank_partner', 'account':'trans_account_partner',
                                    'date':'trans_date','type':'trans_type','operation':'trans_operation',
                                    'k_symbol':'trans_k_symbol'})

# convert transaction date to normal date
trans['trans_date'] = trans.trans_date.apply(get_date)

# convert transaction type to english
trans_type_options = defaultdict(lambda: 'UNKNOWN', 
                           {'PRIJEM': 'CREDIT', 
                            'VYDAJ': 'WITHDRAWAL'})

trans['trans_type'] = trans.trans_type.apply(lambda x: trans_type_options[x])

# convert transaction Operation to english
trans_op_options = defaultdict(lambda: 'UNKNOWN', 
                           {'VYBER KARTOU': 'CC_WITHDRAWAL', 
                            'VKLAD': 'CREDIT_IN_CASH',
                            'PREVOD Z UCTU':'COLLECTION_FROM_OTHER_BANK',
                            'VYBER':'WITHDRAWAL_IN_CASH',
                            'PREVOD NA UCET':'REMITTANCE_TO_OTHER_BANK'})

trans['trans_operation'] = trans.trans_operation.apply(lambda x: trans_op_options[x])

# convert transaction K-symbol to english
trans_k_symbol_options = defaultdict(lambda: 'UNKNOWN', 
                           {'POJISTNE': 'INSURANCE_PAYMENT', 
                            'SLUZBY': 'PAYMENT_FOR_STATEMENT',
                            'UROK': 'INTEREST_CREDITED',
                            'SANKC. UROK': 'SANCTION_INTEREST',
                            'SIPO': 'HOUSEHOLD',
                            'DUCHOD': 'OLD_AGE_PENSION',
                            'UVER': 'LOAN_PAYMENT'})

trans['trans_k_symbol'] = trans.trans_k_symbol.apply(lambda x: trans_k_symbol_options[x])

In [14]:
# WITHDRAWAL_IN_CASH has trans_type UNKNOWN or WITHDRAWAL
# so we can replace all UNKNOWN to WITHDRAWAL
trans.groupby(['trans_operation', 'trans_type'])['trans_id'].count()

trans_operation             trans_type
CC_WITHDRAWAL               WITHDRAWAL      8036
COLLECTION_FROM_OTHER_BANK  CREDIT         65226
CREDIT_IN_CASH              CREDIT        156743
REMITTANCE_TO_OTHER_BANK    WITHDRAWAL    208283
UNKNOWN                     CREDIT        183114
WITHDRAWAL_IN_CASH          UNKNOWN        16666
                            WITHDRAWAL    418252
Name: trans_id, dtype: int64

In [15]:
trans.loc[trans['trans_type'] == 'UNKNOWN', 'trans_type'] = 'WITHDRAWAL'

In [18]:
# Function to create date category
def date_category(date_val):
    if(date_val < datetime.datetime(year=1995, day=1, month=1)):
        return("First_3_years")
    else:
        return("Last_3_years")

trans['trans_date_category'] = trans.trans_date.apply(date_category)

# Function to Recency, Frequency and Monetory aggregated values - Transactional data
def rfm(row): 
    ref_date = datetime.datetime(year=1999, day=1, month=1)
    row['recent_transaction'] = (ref_date - row['trans_date'].max()).days
    row['length_of_relationship'] = (row['trans_date'].max() - row['trans_date'].min()).days
    row['mon_avg_balance'] = row['balance_after_trans'].mean()
    row['freq_transaction'] = row['trans_id'].count()
    
    cred = row.loc[(row.trans_type == 'CREDIT')]
    withdraw = row.loc[(row.trans_type == 'WITHDRAWAL')]
    
    row['mon_trans_cred'] = cred['trans_amount'].mean()
    row['mon_trans_withraw'] = withdraw['trans_amount'].mean()
    
    First_3_years = row.loc[(row.trans_date_category == 'First_3_years')]
    Last_3_years = row.loc[(row.trans_date_category == 'Last_3_years')]
    
    row['freq_first_3years'] = First_3_years['trans_id'].count()
    row['freq_last_3years'] = Last_3_years['trans_id'].count()
    
    return row

trans = trans.groupby('account_id').apply(rfm)

trans_df = trans.drop(['trans_id', 'trans_date', 'trans_date_category','trans_type','trans_operation','trans_amount','balance_after_trans','trans_k_symbol','trans_bank_partner','trans_account_partner'], axis=1).drop_duplicates()

trans_df.to_pickle('data_berka/trans.pkl')
trans_df.to_csv('data_berka/trans.csv')

#### Merging & dumy variables

In [21]:
df = pd.merge(client, disp, on="client_id",how='left')
df = df.merge(card, on="disp_id",how='left')
df = df.merge(district, on="district_id",how='left')
df = df.merge(account, on="account_id",how='left')
df = df.merge(order_df, on="account_id",how='left')
df = df.merge(loan, on="account_id",how='left')
df = df.merge(trans_df, on="account_id",how='left')

#df = pd.get_dummies(df, drop_first=True)

In [22]:
df.head()

,client_id,district_id,client_age,client_gender,disp_id,account_id,disp_type,card_id,card_type,days_since_card_issuance,...,monthly_loan_payment,loan_status,recent_transaction,length_of_relationship,mon_avg_balance,freq_transaction,mon_trans_cred,mon_trans_withraw,freq_first_3years,freq_last_3years
0,1,18,48.04,F,1,1,OWNER,NaN,NaN,NaN,...,NaN,NaN,1,1378,15994.008368,239,1905.102941,1320.102190,0,239
1,2,1,73.91,M,2,2,OWNER,NaN,NaN,NaN,...,3373.0,A,1,2134,36540.704393,478,10506.930921,4768.177301,139,339
2,3,1,78.23,F,3,2,DISPONENT,NaN,NaN,NaN,...,3373.0,A,1,2134,36540.704393,478,10506.930921,4768.177301,139,339
3,4,5,62.08,M,4,3,OWNER,NaN,NaN,NaN,...,NaN,NaN,1,542,26973.506838,117,3762.152174,1717.785915,0,117
4,5,5,58.49,F,5,3,DISPONENT,NaN,NaN,NaN,...,NaN,NaN,1,542,26973.506838,117,3762.152174,1717.785915,0,117


In [23]:
# Identifying attractive customers
df["potential_loan"] = ((df.client_age < 70) # Cut-off old clients from long-term loan propositions
& (df.disp_type == "OWNER") # Only address the decision-makers
& (df.mon_avg_balance > 0) # Select those with a positive cash-flow
& (df.mon_trans_withraw > 0.5*df.mon_trans_cred) # Select those who spend at least half of what they earn (cut-off big savers)
& (df.loan_status != "B") & (df.loan_status != "C") & (df.loan_status != "D") # Select those who never borrowed | those who did and paid back
& (df.freq_first_3years < df.freq_last_3years)) # Select the ones who show an increase of activity recently

# Identifying risky customers
df["risky_customer"] = ((df.loan_status == "B") | (df.loan_status == "D") # Identify customers having debt issues
                        | (df.mon_avg_balance < 0)) # Identify customers with risky cash-flow
                        

df["type_of_customer"] = "regular"
df["type_of_customer"][df["potential_loan"]] = "prospect"
df["type_of_customer"][df["risky_customer"]] = "risky"

C:\Users\SIDDY\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\SIDDY\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [24]:
df.to_pickle('data_berka/basetable.pkl')
df.to_csv('data_berka/basetable.csv')